_Control

In [ ]:
%reset -f

In [151]:
#import
import os
import numpy as np
import sympy as sp
import matplotlib as mpl
import pandas as pd
import jax
import jax.numpy as jnp

dspath="./datasets"

In [ ]:
def dummyf(x):
	return x**2

_Notes

In [ ]:
# matplotlib
import matplotlib.pyplot as plt
import numpy as np

"""
for i in range(decoded.shape[0]):

    array=decoded[i,0,:,:,0]
    
    # Plot the array as a grayscale image
    plt.imshow(array, cmap='gray')

    plt.colorbar()  # Add a color bar to the side for reference
    plt.title('Grayscale Image of 128x128 Array')
    plt.show()
"""

fig, axes = plt.subplots(10, 6, figsize=(12, 15))
for i in range(10):
	for j in range(6):
		array=decoded[6*i+j,0,:,:,0]
		axes[i, j].imshow(array, cmap='gray')
		axes[i, j].set_title(f"Plot {6*i+j+1}")
plt.tight_layout()
plt.show()

In [ ]:
# jax checkpoints
import jax.numpy as jnp
from flax.training import checkpoints
import os
ckpt_dir = os.getcwd()+"/checkpoints/TEST"

restored={}
state = {'a':jnp.ones(16)}
#restored = checkpoints.restore_checkpoint(ckpt_dir, state, prefix=f"_ARRAY_")

checkpoints.save_checkpoint(ckpt_dir, state, 1, prefix=f"_ARRAY_", keep=5, overwrite=True)

In [ ]:
# basic TrainState
import flax.linen as nn
from flax.training.train_state import TrainState
import jax, jax.numpy as jnp
import optax

x = jnp.ones((1,2))
y = jnp.ones((1,2))
z = jnp.array([[1,1],[-2,4],[0,3]])
model = nn.Dense(2)
variables = model.init(jax.random.key(0), x)
tx = optax.adam(2e-2)

state=TrainState.create(apply_fn=model.apply, params=variables, tx=tx)

#print(state.params['params']['kernel'])
#print(model.apply(state.params,z))

def loss(params, x, y):
	predictions=state.apply_fn(params,x)
	return optax.l2_loss(predictions,y).mean()
print(loss(state.params,x,y))

for _ in range(300):
	
	grads=jax.grad(loss)(state.params,x,y)
	state=state.apply_gradients(grads=grads)
	#display(loss(state.params,x,y))

print(state.params)
print(loss(state.params,x,y))

In [ ]:
# transformers pipeline
# long loading time
from transformers import pipeline

'''# Test a pre-trained pipeline (e.g., sentiment-analysis)
classifier = pipeline(task='sentiment-analysis',device='cuda')
print(classifier("Hello World"))
print(classifier(["Hugging Face is amazing!","Red Blue yellow"]))

generator = pipeline("text-generation", model="gpt2", device='cuda')
print(generator("Once upon a time", max_length=50, num_return_sequences=1))
print(generator("I've always been shy growing up.", max_length=250, num_return_sequences=1))'''


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens1 = tokenizer("Hello, how are you?", return_tensors="pt")
tokens2 = tokenizer("Hello", return_tensors="pt")
print(tokens1)
print(tokens2)

In [ ]:
# PyTorch BatchNorm evaluation mode issue
import torch
import torch.nn as nn
import numpy as np

# Define a BatchNorm layer
bn = nn.BatchNorm1d(3,momentum=0.1)

# Simulate training mode
bn.train()  # Training mode
input_train = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
output_train = bn(input_train)
print("Training Mode Output:", output_train)

'''mean_rows = np.mean(np.array(input_train), axis=0)

# Variance along columns (axis=0)
variance_cols = np.var(np.array(input_train), axis=0)

print("Mean along rows:", mean_rows)
print("Variance along columns:", variance_cols)'''


'''print([pn, p] for pn,p in bn.named_parameters())
for pn,p in bn.named_parameters():
	print([pn,p])'''
print(bn.state_dict())

# Switch to evaluation mode
bn.eval()  # Evaluation mode
input_eval = torch.tensor([[4.0, 1.0, 6.0],[7.0, 8.0, 9.0]])
output_eval = bn(input_eval)
print("Evaluation Mode Output:", output_eval)
print(bn.state_dict())

input_eval = torch.tensor([[7.0, 8.0, 9.0]])
output_eval = bn(input_eval)
print("Evaluation Mode Output:", output_eval)
print(bn.state_dict())

Training Mode Output: tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]], grad_fn=<NativeBatchNormBackward0>)
OrderedDict([('weight', tensor([1., 1., 1.])), ('bias', tensor([0., 0., 0.])), ('running_mean', tensor([0.4000, 0.5000, 0.6000])), ('running_var', tensor([1.8000, 1.8000, 1.8000])), ('num_batches_tracked', tensor(1))])
Evaluation Mode Output: tensor([[2.6833, 0.3727, 4.0249],
        [4.9193, 5.5902, 6.2610]], grad_fn=<NativeBatchNormBackward0>)
OrderedDict([('weight', tensor([1., 1., 1.])), ('bias', tensor([0., 0., 0.])), ('running_mean', tensor([0.4000, 0.5000, 0.6000])), ('running_var', tensor([1.8000, 1.8000, 1.8000])), ('num_batches_tracked', tensor(1))])
Evaluation Mode Output: tensor([[4.9193, 5.5902, 6.2610]], grad_fn=<NativeBatchNormBackward0>)
OrderedDict([('weight', tensor([1., 1., 1.])), ('bias', tensor([0., 0., 0.])), ('running_mean', tensor([0.4000, 0.5000, 0.6000])), ('running_var', tensor([1.8000, 1.800

In [149]:
import torch
import torch.nn as nn
import numpy as np

x = torch.tensor(4.0, requires_grad=True)
y = x ** 2  # y = x²
y.retain_grad()
z = torch.sin(y)

print(z)
print(y)
print(x)

z.backward()  # Compute gradients
print(x.grad)  # Output: 4.0 (dy/dx = 2*x, at x=2)
print(y.grad)
print(z.grad)

tensor(-0.2879, grad_fn=<SinBackward0>)
tensor(16., grad_fn=<PowBackward0>)
tensor(4., requires_grad=True)
tensor(-7.6613)
tensor(-0.9577)
None


/tmp/ipykernel_251841/420265378.py:17: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(z.grad)


In [ ]:
import torch
from torch.utils.checkpoint import checkpoint

# Define a function to be checkpointed
def forward_func(input):
    return torch.relu(input)
#pdb.set_trace()

# Input tensor
input_tensor = torch.randn(5, requires_grad=True)

# Use checkpoint
output = checkpoint(forward_func, input_tensor)

# Backward pass
z=output.sum()
z.backward()
print(input_tensor.grad)

tensor([1., 0., 0., 1., 1.])


In [ ]:
import pdb

for i in range(10):
	pdb.set_trace()
	print("{i} th loop", i)

None
> /tmp/ipykernel_446290/1839701227.py(5)<module>()
      1 import pdb
      2 
      3 for i in range(10):
      4         pdb.set_trace()
----> 5         print("{i} th loop", i)

--Call--
> /home/mike_1102/miniconda3/envs/env02/lib/python3.11/site-packages/ipykernel/iostream.py(655)write()
    653                 )
    654 
--> 655     def write(self, string: str) -> Optional[int]:  # type:ignore[override]
    656         """Write to current stream after encoding if necessary
    657 

> /home/mike_1102/miniconda3/envs/env02/lib/python3.11/site-packages/ipykernel/iostream.py(664)write()
    662 
    663         """
--> 664         parent = self.parent_header
    665 
    666         if not isinstance(string, str):

--Call--
> /home/mike_1102/miniconda3/envs/env02/lib/python3.11/site-packages/ipykernel/iostream.py(505)parent_header()
    503                 raise ValueError(msg)
    504 
--> 505     @property
    506     def parent_header(self):
    507         try:

> /home/mike_

In [8]:
import pdb

def divide(a, b):
    #pdb.set_trace()  # Debugger starts here
    return a / b

divide(10, 0)

ZeroDivisionError: division by zero

In [10]:
%debug

> /tmp/ipykernel_446290/2331031421.py(5)divide()
      3 def divide(a, b):
      4     #pdb.set_trace()  # Debugger starts here
----> 5     return a / b
      6 
      7 divide(10, 0)

10
10
0
0
0
0
0
0
0
0
0
0
0
0
0
0
